Heavily inspired by the lesson 
https://learn.deeplearning.ai/langchain-chat-with-your-data/lesson/2/document-loading
I want to:
1. Get the contents of a git repo with lesson material (in github-pages).
2. Add the files with content (for now only the markdown files) and create a vector store.
3. Be able to chat with the material, ask questions like: 'Tell me about the concept blablathisnthat ...', or 'Ask a question about ...'. 

In [ ]:
# These should already be handled by the requirements.txt
#!pip install python-dotenv
#!pip install pypdf 
#!pip install markdown
#!pip install openai
#!pip install GitPython
#!pip install bs4
#!pip list

In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.document_loaders import PyPDFLoader
#loader = PyPDFLoader("samples/tutorial_Class.pdf")
loader = PyPDFLoader("samples/tutorial_Class.pdf")
pages = loader.load()

In [3]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://github.com/basecamp/handbook/blob/master/37signals-is-you.md")

In [4]:
pages[0]

Document(page_content='Stéphane Ducasse, Lukas Renggli, David C. Shaffer, Rick Zacconewith Michael DaviesDynamic WebDevelopmentwith', metadata={'source': 'samples/tutorial_Class.pdf', 'page': 0})

In [5]:
# Git Clone 
import subprocess

repo_url = "https://github.com/stasemsoft/softwarematerial.git"
local_dir = ".git_softwaremat"

subprocess.check_call(["git", "clone", repo_url, local_dir])


Cloning into '.git_softwaremat'...


0

In [6]:
import pathlib

repo_path = pathlib.Path(local_dir)
markdown_files = list(repo_path.glob("**/*.md"))
markdown_files[0:4]


[PosixPath('.git_softwaremat/README.md'),
 PosixPath('.git_softwaremat/objects.md'),
 PosixPath('.git_softwaremat/docs/legacy/Explanation-Array-Lists.md'),
 PosixPath('.git_softwaremat/docs/legacy/readme.md.md')]

In [7]:
from langchain.document_loaders import NotionDirectoryLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
import numpy as np

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

# 1 embedding to put all text in. 
embedding = OpenAIEmbeddings()
embeddings = []

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)



def SplitMarkdownText(markdown_text):
    md_header_splits = markdown_splitter.split_text(markdown_text)
    global embeddings.append(embedding.embed_query(md_header_splits))
    print(np.dot(embeddings[0:], embeddings[:0]))
    return md_header_splits


SyntaxError: invalid syntax (2531106602.py, line 22)

In [9]:
for markdown_file in markdown_files[0:3]:
    with open(markdown_file, "r") as f:
        print("--------------------")
        content = f.read()
        chunk = SplitMarkdownText(content)
        print(chunk)


--------------------


UnboundLocalError: local variable 'embeddings' referenced before assignment